In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square


In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.4))
fes = H1(mesh, order=4, dirichlet='right|bottom')
u = fes.TrialFunction()
v = fes.TestFunction()

In [4]:
a = BilinearForm(fes, flags={"eliminate_internal":True})
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()

u = GridFunction(fes)

In [5]:
f.vec.data += a.harmonic_extension_trans * f.vec

In [6]:
u.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(coupling=True))*f.vec

In [8]:
u.vec.data += a.inner_solve * f.vec

In [9]:
u.vec.data += a.harmonic_extension * u.vec

In [10]:
Draw(u)

In [11]:
for i in range(fes.ndof):
    print(fes.CouplingType(i))

COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUP

In [13]:
doftypes = {}
for i in range(fes.ndof):
    type = fes.CouplingType(i)
    if type in doftypes.keys():
        doftypes[type] += 1
    else:
        doftypes[type] = 1
doftypes

{COUPLING_TYPE.LOCAL_DOF: 54,
 COUPLING_TYPE.INTERFACE_DOF: 99,
 COUPLING_TYPE.WIREBASKET_DOF: 16}